In [59]:
import pandas as pd
from bs4 import BeautifulSoup
import codecs

In [60]:
date_list=[]
open_list=[]
high_list=[]
low_list=[]
close_list=[]
adjclose_list=[]
volume_list=[]
df=pd.DataFrame()

In [61]:
def scrape():
  file = codecs.open("html-file.html", "r", "utf-8")
  soup = BeautifulSoup(file, 'html.parser') 
  tr=soup.findAll('tr',attrs={'class':'BdT Bdc($seperatorColor) Ta(end) Fz(s) Whs(nw)'})
  for i in tr:
    try:
      date=i.find('td',attrs={'class':'Py(10px) Ta(start) Pend(10px)'})
      date_list.append(date.text)
      new_column=i.findAll('td',attrs={'class':'Py(10px) Pstart(10px)'})
      s=0
      for j in new_column:
        if(s==0):
          open_list.append(j.text)
          s=s+1
        elif(s==1):
          high_list.append(j.text)
          s=s+1
        elif(s==2):
          low_list.append(j.text)
          s=s+1
        elif(s==3):
          close_list.append(j.text)
          s=s+1
        elif(s==4):
          adjclose_list.append(j.text)
          s=s+1
        else:
          volume_list.append(j.text)
    except:
      continue    
scrape()

In [62]:
df['date']=date_list
df['open']=open_list
df['high']=high_list
df['low']=low_list
df['close']=close_list
df['Adj close']=adjclose_list
df['volume']=volume_list
df

,date,open,high,low,close,Adj close,volume
0,"Oct 07, 2022",142.54,143.10,139.45,140.09,140.09,"85,859,100"
1,"Oct 06, 2022",145.81,147.54,145.22,145.43,145.43,"68,402,200"
2,"Oct 05, 2022",144.07,147.38,143.01,146.40,146.40,"79,471,000"
3,"Oct 04, 2022",145.03,146.22,144.26,146.10,146.10,"87,830,100"
4,"Oct 03, 2022",138.21,143.07,137.69,142.45,142.45,"114,311,700"
...,...,...,...,...,...,...,...
2511,"Oct 15, 2012",22.58,22.68,22.28,22.67,19.44,"432,502,000"
2512,"Oct 12, 2012",22.48,22.69,22.33,22.49,19.28,"460,014,800"
2513,"Oct 11, 2012",23.09,23.11,22.43,22.43,19.24,"546,081,200"
2514,"Oct 10, 2012",22.85,23.03,22.75,22.89,19.63,"510,356,000"


TIME SERIES FORECASTING

In [63]:
import math
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

In [64]:
df['date'] = pd.to_datetime(df['date'])

In [65]:
df

,date,open,high,low,close,Adj close,volume
0,2022-10-07,142.54,143.10,139.45,140.09,140.09,"85,859,100"
1,2022-10-06,145.81,147.54,145.22,145.43,145.43,"68,402,200"
2,2022-10-05,144.07,147.38,143.01,146.40,146.40,"79,471,000"
3,2022-10-04,145.03,146.22,144.26,146.10,146.10,"87,830,100"
4,2022-10-03,138.21,143.07,137.69,142.45,142.45,"114,311,700"
...,...,...,...,...,...,...,...
2511,2012-10-15,22.58,22.68,22.28,22.67,19.44,"432,502,000"
2512,2012-10-12,22.48,22.69,22.33,22.49,19.28,"460,014,800"
2513,2012-10-11,23.09,23.11,22.43,22.43,19.24,"546,081,200"
2514,2012-10-10,22.85,23.03,22.75,22.89,19.63,"510,356,000"


In [66]:
df=df.set_index('date')

In [67]:
df

,open,high,low,close,Adj close,volume
date,,,,,,
2022-10-07,142.54,143.10,139.45,140.09,140.09,"85,859,100"
2022-10-06,145.81,147.54,145.22,145.43,145.43,"68,402,200"
2022-10-05,144.07,147.38,143.01,146.40,146.40,"79,471,000"
2022-10-04,145.03,146.22,144.26,146.10,146.10,"87,830,100"
2022-10-03,138.21,143.07,137.69,142.45,142.45,"114,311,700"
...,...,...,...,...,...,...
2012-10-15,22.58,22.68,22.28,22.67,19.44,"432,502,000"
2012-10-12,22.48,22.69,22.33,22.49,19.28,"460,014,800"
2012-10-11,23.09,23.11,22.43,22.43,19.24,"546,081,200"


In [69]:
data=df.filter(['close'])
dataset=data.values
training_data_len=math.ceil(len(dataset)* .8 )
training_data_len

2013

In [70]:
scaler=MinMaxScaler(feature_range=(0,1))
scaled_data= scaler.fit_transform(dataset)
scaled_data

array([[0.75056527],
       [0.78233964],
       [0.78811139],
       ...,
       [0.05045817],
       [0.05319529],
       [0.05212424]])

In [71]:
train_data=scaled_data[0:training_data_len, :]
x_train=[]
y_train=[]
for i in range(60,len(train_data)):
  x_train.append(train_data[i-60:i,0])
  y_train.append(train_data[i,0])
  if (i<=61):
    print(x_train)
    print(y_train)

[array([0.75056527, 0.78233964, 0.78811139, 0.78632631, 0.76460788,
       0.73931929, 0.76478639, 0.80858027, 0.82000476, 0.81411401,
       0.81209092, 0.82583601, 0.83166726, 0.85058908, 0.83618946,
       0.81369749, 0.82363442, 0.84112817, 0.83238129, 0.88944425,
       0.8533857 , 0.83607045, 0.84499583, 0.83648697, 0.8441033 ,
       0.85689635, 0.85249316, 0.86254909, 0.87724622, 0.89057479,
       0.92871593, 0.9138403 , 0.91205522, 0.91407831, 0.93758182,
       0.95323099, 0.95561109, 0.9465667 , 0.94751874, 0.94103296,
       0.91955254, 0.92401523, 0.89831013, 0.89801261, 0.90086874,
       0.90360586, 0.90550994, 0.86909437, 0.87801975, 0.88397001,
       0.85326669, 0.84993455, 0.81905272, 0.82708556, 0.83386886,
       0.84136618, 0.82762109, 0.81548257, 0.79209806, 0.81054385])]
[0.8004284184219922]
[array([0.75056527, 0.78233964, 0.78811139, 0.78632631, 0.76460788,
       0.73931929, 0.76478639, 0.80858027, 0.82000476, 0.81411401,
       0.81209092, 0.82583601, 0.8316

In [72]:
x_train, y_train= np.array(x_train), np.array(y_train)

In [74]:
x_train= np.reshape(x_train, (x_train.shape[0], x_train.shape[1] ,1))
x_train.shape

(1953, 60, 1)

In [75]:
model= Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(x_train.shape[1],1))) 
model.add(LSTM(50, return_sequences=False))
model.add(Dense(25))
model.add(Dense(1))

In [76]:
model.compile(optimizer='adam',loss='mean_squared_error')

In [78]:
model.fit(x_train,y_train,batch_size=1, epochs=1)

1953/1953 [==============================] - 33s 16ms/step - loss: 0.0016


In [79]:
test_data= scaled_data[training_data_len - 60:, :]
x_test=[]
y_test=dataset[training_data_len:, :]
for i in range(60,len(test_data)):
  x_test.append(test_data[i-60:i,0])

In [80]:
x_test= np.array(x_test)

In [81]:
x_test= np.reshape(x_test, (x_test.shape[0], x_test.shape[1],1))

In [82]:
predictions=model.predict(x_test)
predictions= scaler.inverse_transform(predictions)

In [87]:
valid

,close,predictions
date,,
2014-10-09,25.25,25.187204
2014-10-08,25.20,25.202818
2014-10-07,24.69,25.246243
2014-10-06,24.91,25.256968
2014-10-03,24.91,25.264414
...,...,...
2012-10-15,22.67,22.725294
2012-10-12,22.49,22.875383
2012-10-11,22.43,22.963062


As we can see here, the values are so close to each other. We can tell that the model is functioning pretty well